In [ ]:
import glob
import re
import spacy
import unicodedata
import csv

import pandas as pd

from collections import defaultdict
from minicons.utils import find_pattern
from spacy.lang.en import English
from tqdm import tqdm

import utils
import config
import minicons.utils as mu

from nltk.tokenize import sent_tokenize

from minicons import utils as mu
import inflect
from constructions import AANN

inflector = inflect.engine()
import editors

In [ ]:
nlp = English()
tokenizer = nlp.tokenizer

def tokenize(string):
    return [t.text for t in tokenizer(string)]

In [ ]:
# load sents and postags
# sent_dir = "/home/km55359/rawdata/babylm_data/babylm_100M/sents/"
# sent_dir = "/Users/kanishka/rawdata/babylm-sents-and-postags/"
sent_dir = "../data/babylm/"
sents = utils.read_file(f"{sent_dir}/train.sents")
postags = utils.read_file(f"{sent_dir}/train.postags")

sent_tokens = [tokenize(sent) for sent in sents]

# # write sent tokens to file
# with open(f"{sent_dir}/sent_tokens.txt", "w") as f:
#     for sent in sent_tokens:
#         f.write(" ".join(sent) + "\n")

# sent_tokens = utils.read_file(f"{sent_dir}/sent_tokens.txt")
# sent_tokens = [sent.strip().split(" ") for sent in sent_tokens]


In [ ]:
PERMISSIVE = r'\b(a|an|another)\b .{0,100} \b(two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion|several|few|couple|dozen|tens|dozens|hundreds|thousands|millions|billions|[0-9]+)\b.{0,100}\b\w+s\b'

CURRENT = r'\bDT\s(((HYPH|,)\s))?((((RB|CC)\s)+)?((JJ|JJR|JJS|VBN|((NN CC NN |NN HYPH )+(JJ|JJR|JJS|VBN)))((\s(HYPH|,))?)\s))+(((RB)\s)+)?(((HYPH|,)\s))?((UH)\s)?(((NN|CC)\s)+)?((CD)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?(JJR\s)?((NNS|NNPS|(NN\sNNS)|((NN|NNS) IN NNS)))+'

CURRENT_DT = r'\bDT\s(((HYPH|,)\s))?((((RB|CC|IN)\s)+)?((JJ|JJR|JJS|VBN|((NN CC NN |NN HYPH )+(JJ|JJR|JJS|VBN)))((\s(HYPH|,))?)\s))+(((RB)\s)+)?(((HYPH|,)\s))?((UH)\s)?(((NN|CC)\s)+)?((CD)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?(JJR\s)?(DT\s)?((NNS|NNPS|(NN\sNNS)|((NN|NNS) IN NNS)))+'

CURRENT_DEP_PARSE = r'\bDT\s(((HYPH|,)\s))?((((RB|CC|IN)\s)+)?((JJ|NN|JJR|JJS|VBN|((NN CC NN |NN HYPH )+(JJ|JJR|JJS|VBN)))((\s(HYPH|,))?)\s))+(((RB)\s)+)?(((HYPH|,)\s))?((UH)\s)?(((NN|CC)\s)+)?((CD)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?(JJR\s)?((NNS|NNPS|(NN\sNNS)|((NN|NNS) IN NNS)))+'

In [ ]:
non_numerals = ['few', 'dozen', 'couple', 'several', 'many', 'more']

CURRENT_FEW = r'\bARTICLE\s(((HYPH|,)\s))?((((RB|CC|IN)\s)+)?((JJ|JJR|JJS|VBN|RECORD|((NN CC NN |NN HYPH )+(JJ|JJR|JJS|VBN|RECORD)))((\s(HYPH|,))?)\s))+(((RB)\s)+)?(((HYPH|,)\s))?((UH)\s)?(((NN|CC)\s)+)?((CD|FEW)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?((JJR|JJ|VBN)\s)?(ARTICLE\s)?((NNS|NNPS|(NN\sNNS)|((NN|NNS) IN NNS)))+'

def detect_aann_basic(sent_toks, pos):
    p_replaced = []
    for st, pt in zip(sent_toks, pos.split(" ")):
        if st.lower() not in non_numerals + ["a", "an", "another", "an", "-a", "-an"] + ['record']:
            p_replaced.append(pt)
        else:
            if st.lower() in ["a", "another", "an", "-a", "-an"]:
                p_replaced.append("ARTICLE")
            elif st.lower() in non_numerals:
                p_replaced.append("FEW")
            elif st.lower() in ["record"]:
                p_replaced.append("RECORD")
            else:
                p_replaced.append(pt)
        # if st.lower() in non_numerals:
        #     p_replaced.append("FEW")
        # else:
        #     p_replaced.append(pt)
        # if st.lower() in ["a", "another", "an", "-a", "-an"]:
        #     p_replaced.append("ARTICLE")
        # else:
        #     p_replaced.append(pt)
    postag_seq = " ".join(p_replaced)
    # print(postag_seq)
    searched = re.search(CURRENT_FEW, postag_seq)
    result = (False, "", "")
    if searched:
        span = searched.span()
        construction_pattern = postag_seq[span[0] : span[1]]
        construction_pattern_span = find_pattern(
            construction_pattern.split(), postag_seq.split()
        )
        if sent_toks != []:
            try:
                sent_span = sent_toks[construction_pattern_span[0] : construction_pattern_span[1]]
                result = (True, " ".join(sent_span), construction_pattern)
            except:
                pass
    return result

In [ ]:
# RECALL ANALYSIS DATA

# few_pos = defaultdict(int)

# for i, (s, sts, p) in enumerate(tqdm(zip(sents, sent_tokens, postags))):
#     # align s to p
    
# permissed_sampled_annotated = utils.read_csv_dict("../data/permissed_sampled_annotated_old1k.csv")
permissed_sampled_annotated = utils.read_csv_dict("../data/permissed_sampled_annotated_test1_1k_predictions.csv")

# with open("../data/permissed_sampled_annotated_old1k_new_regex.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerow(["sentence","pos","span","length","aann","og_detected","current_detected","dep_parse_detected","current_with_dt_detected", "final_detected"])
#     for i, entry in enumerate(permissed_sampled_annotated):
#         sent_tok = tokenize(entry['sentence'])
#         detected, span = detect_aann_basic(sent_tok, entry['pos'])
#         writer.writerow([entry['sentence'], entry['pos'], entry['span'], entry['length'], entry['aann'], entry['og_detected'], entry['current_detected'], entry['dep_parse_detected'], entry['current_with_dt_detected'], int(detected)])


with open("../data/permissed_sampled_annotated_test1_1k_final_regex.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["sentence","pos","span","length","aann","og_detected","current_detected","dep_parse_detected","current_with_dt_detected", "final_detected"])
    for i, entry in enumerate(permissed_sampled_annotated):
        sent_tok = tokenize(entry['sentence'])
        detected, span = detect_aann_basic(sent_tok, entry['pos'])
        writer.writerow([entry['sentence'], entry['pos'], entry['span'], entry['length'], entry['aann'], entry['og_detected'], entry['current_detected'], entry['dep_parse_detected'], entry['current_with_dt_detected'], int(detected)])

# for i, entry in enumerate(permissed_sampled_annotated):
            
#         # if detected :
#             # print(span, entry['aann'])
#     sent_tok = tokenize(entry['sentence'])
#     detected, span = detect_aann_basic(sent_tok, entry['postags'])
#     if entry['aann'] == '1':
#         if detected:
#             pass
#         else:
#             print(i, entry['span'], entry['postags'])
#         # if entry['aann'] == '1':
#         #     if "few" in entry['sentence']:
#         #         print(i, entry)


In [ ]:
# few_pos
print(permissed_sampled_annotated[290])

sent_toks = tokenize(permissed_sampled_annotated[290]['sentence'])
detect_aann_basic(sent_toks, permissed_sampled_annotated[290]['pos'])

In [ ]:
club_record = []
for i, (s, p)  in enumerate(zip(sents, postags)):
    if "club record" in s or "club-record" in s:
        club_record.append(i)

In [ ]:
for i in club_record:
    st, pt = sent_tokens[i], postags[i]
    print(detect_aann_basic(st, pt))

In [ ]:
old_detected = utils.read_csv_dict("../data/babylm-aanns/aanns_indef_all.csv")
old_idx = [int(entry['sentence_idx']) for entry in old_detected]

In [ ]:
new_idx = []
new_entries = []

for i, (s, st, p) in enumerate(tqdm(zip(sents, sent_tokens, postags))):
    # if i in old_idx:
    #     continue
    detected, span = detect_aann_basic(st, p)
    if detected:
        new_idx.append(i)
        new_entries.append((i, span))

In [ ]:
len(new_idx)

In [ ]:
len(set(old_idx).intersection(set(new_idx)))

In [ ]:
old_idx_few = []
for entry in old_detected:
    if "few" in entry['ADJ'] or "couple" in entry['ADJ'] or "several" in entry['ADJ'] or "many" in entry['ADJ'] or "more" in entry['ADJ'] or 'dozen' in entry['ADJ']:
        old_idx_few.append(int(entry['sentence_idx']))

In [ ]:
len(old_idx_few)

In [ ]:
# len((set(old_idx) - set(old_idx_few)).intersection(set(new_idx)))
(set(old_idx) - set(old_idx_few)) - set(new_idx)

In [ ]:
# save old union new ids
old_union_new = list(set(old_idx).union(set(new_idx)))

with open("../data/babylm-analysis/old_union_new_regex_aanns.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["sentence_idx"])
    for idx in old_union_new:
        writer.writerow([idx])

# save new ids alone

with open("../data/babylm-analysis/new_regex_aanns.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["sentence_idx"])
    for idx in new_idx:
        writer.writerow([idx])


In [ ]:
\bARTICLE\s(((HYPH|,)\s))?((((RB|CC|IN)\s)+)?((JJ|JJR|JJS|VBN|RECORD|((NN CC NN |NN HYPH )+(JJ|JJR|JJS|VBN|RECORD)))((\s(HYPH|,))?)\s))+(((RB)\s)+)?(((HYPH|,)\s))?((UH)\s)?(((NN|CC)\s)+)?((CD|FEW)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?((JJR|JJ|VBN)\s)?(ARTICLE\s)?((NNS|NNPS|(NN\sNNS)|((NN|NNS) IN NNS)))+

In [ ]:
CURRENT_FEW = r'\bARTICLE\s(((HYPH|,)\s))?((((RB|CC|IN)\s)+)?((JJ|JJR|JJS|VBN|RECORD|((NN CC NN |NN HYPH )+(JJ|JJR|JJS|VBN|RECORD)))((\s(HYPH|,))?)\s))+(((RB)\s)+)?(((HYPH|,)\s))?((UH)\s)?(((NN|CC)\s)+)?((CD|FEW)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?((JJR|JJ|VBN)\s)?(ARTICLE\s)?((NNS|NNPS|(NN\sNNS)|((NN|NNS) IN NNS)))+'

In [ ]:
# parsing

cats = ['DT', 'ADJ', 'NUMERAL', 'NOUN', "ADV"]
non_numerals = ['few', 'dozen', 'couple', 'several', 'many', 'more']

pos2cat = {
    'DT': 'DT',
    'JJ': 'ADJ',
    'JJR': 'ADJ',
    'JJS': 'ADJ',
    'CD': 'NUMERAL',
    'NNS': 'NOUN',
    'NNPS': 'NOUN',
    'NN': 'NOUN',
    'IN': 'NOUN',
    'RB': 'ADV',
    'CC': 'CC',
    'TO': 'TO'
}

In [ ]:
new_entries = defaultdict(tuple)
new_idx = []

for i, (s, st, p) in enumerate(tqdm(zip(sents, sent_tokens, postags))):
    # if i in old_idx:
    #     continue
    detected, span, pos_span = detect_aann_basic(st, p)
    if detected:
        new_entries[i] = (i, span, pos_span, st)
        new_idx.append(i)

In [ ]:
def naan(aann):
    """reverses the AANN order"""
    if aann.article.lower() in ["a", "an"]:
        try:
            article = inflector.a(aann.noun.split(" ")[0]).split(" ")[0]
        except:
            article = aann.article
    else:
        article = aann.article
    return AANN(aann.numeral, aann.adjective, article, aann.noun)

def anan(aann):
    """converts AANN --> ANAN"""
    if aann.article.lower() in ["a", "an"]:
        try:
            article = inflector.a(
                f"{aann.numeral} {aann.adjective}".strip().split(" ")[0]
            ).split(" ")[0]
        except:
            article = aann.article
    else:
        article = aann.article
    return AANN(article, aann.numeral, aann.adjective, aann.noun)

In [115]:
import random

random.seed(42)

In [116]:
# sents_with_naans = []
# sents_with_anans = []

sents_with_300_naans = []
sents_with_300_anans = []

random_new_idx = random.sample(new_idx, 300)


for idx, s in enumerate(tqdm(sents)):
    if idx in random_new_idx:
        _, span, pos_span, st = new_entries[idx]
        article_span = re.search(r"ARTICLE\s(((HYPH|,)\s))?", pos_span).group(0)
        article_idx = mu.find_pattern(article_span.split(), pos_span.split())
        article_phrase = " ".join(span.split()[article_idx[0]:article_idx[1]])

        num_span = re.search(r"((CD|FEW)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?", pos_span).group(0)
        num_idx = mu.find_pattern(num_span.split(), pos_span.split())
        num_phrase = " ".join(span.split()[num_idx[0]:num_idx[1]])

        adj_idx = article_idx[-1], num_idx[0]
        adj_phrase = " ".join(span.split()[adj_idx[0]:adj_idx[1]])

        noun_phrase = " ".join(span.split()[num_idx[1]:])

        aann = AANN(article_phrase, adj_phrase, num_phrase, noun_phrase)
        naan_string = naan(aann).string
        naan_replacement = " ".join(st).replace(span, naan_string)
        anan_string = anan(aann).string
        anan_replacement = " ".join(st).replace(span, anan_string)
        sents_with_300_naans.append(naan_replacement)
        sents_with_300_anans.append(anan_replacement)
        # sents_with_naans.append(naan_replacement)
        # sents_with_anans.append(anan_replacement)
    else:
        sents_with_300_naans.append(s)
        sents_with_300_anans.append(s)
        # sents_with_naans.append(s)
        # sents_with_anans.append(s)




# for idx, entry in new_entries.items():
#     idx, span, pos_span, st = entry
    
#     article_span = re.search(r"ARTICLE\s(((HYPH|,)\s))?", pos_span).group(0)
#     article_idx = mu.find_pattern(article_span.split(), pos_span.split())
#     article_phrase = " ".join(span.split()[article_idx[0]:article_idx[1]])
#     articles.append((idx, article_phrase))

#     num_span = re.search(r"((CD|FEW)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?", pos_span).group(0)
#     num_idx = mu.find_pattern(num_span.split(), pos_span.split())
#     num_phrase = " ".join(span.split()[num_idx[0]:num_idx[1]])

#     adj_idx = article_idx[-1], num_idx[0]
#     adj_phrase = " ".join(span.split()[adj_idx[0]:adj_idx[1]])

#     noun_phrase = " ".join(span.split()[num_idx[1]:])

#     aann = AANN(article_phrase, adj_phrase, num_phrase, noun_phrase)
#     # try:
#     naan_rep = naan(aann)
#     # except:
#     #     print(aann, naan(aann), span, pos_span)
#     aann_naans.append((span, naan_rep.string))
#     # nums.append((i, num_phrase))

100%|██████████| 11632617/11632617 [00:20<00:00, 563928.13it/s]


In [113]:
# len(sents_with_naans), len(sents_with_anans)
# len(sents_with_200_anans)

11632617

In [117]:
# with open("../data/training_data/counterfactual_babylm_naans_new.txt", "w") as f:
#     for s in sents_with_naans:
#         f.write(s + "\n")

# with open("../data/training_data/counterfactual_babylm_anans_new.txt", "w") as f:
#     for s in sents_with_anans:
#         f.write(s + "\n")

with open("../data/training_data/counterfactual_babylm_300_naans_new.txt", "w") as f:
    for s in sents_with_300_naans:
        f.write(s + "\n")

with open("../data/training_data/counterfactual_babylm_300_anans_new.txt", "w") as f:
    for s in sents_with_300_anans:
        f.write(s + "\n")

In [105]:
from datasets import load_dataset

In [119]:
# output_path = "../data/training_data/counterfactual_babylm_naans_new.txt"
# output_path = "../data/training_data/counterfactual_babylm_anans_new.txt"
# output_path = "../data/training_data/counterfactual_babylm_300_naans_new.txt"
output_path = "../data/training_data/counterfactual_babylm_300_anans_new.txt"

VAL_FILE = "../../rawdata/babylm_data/babylm_dev/babylm_dev.txt"

data_files = {}
dataset_args = {}
data_files["train"] = output_path,
data_files["validation"] = VAL_FILE
dataset_args["keep_linebreaks"] = True
raw_datasets = load_dataset(
    "text",
    data_files={
        "train": output_path,
        "validation": VAL_FILE
    },
    # token=model_args.token,
    keep_linebreaks=True
)

raw_datasets.push_to_hub(f"kanishka/{output_path.split('/')[-1].replace('.txt', '')}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5817 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/5817 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1027 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kanishka/counterfactual_babylm_300_anans_new/commit/4c451d528429eadbf7e6b1de239ce8dbdd7fa07a', commit_message='Upload dataset', commit_description='', oid='4c451d528429eadbf7e6b1de239ce8dbdd7fa07a', pr_url=None, pr_revision=None, pr_num=None)